In [ ]:
import cv2
import os

# Define keypoint connections and colors
skeleton_connections = [
    # Left side (green)
    ((0, 4), 'green'),  # hip - left hip
    ((4, 5), 'green'),  # left hip - left knee
    ((5, 6), 'green'),  # left knee - left foot
    ((11, 12), 'green'),  # left shoulder - left elbow
    ((12, 13), 'green'),  # left elbow - left wrist
    ((11, 8), 'green'),  # left shoulder - thorax

    # Right side (blue)
    ((0, 1), 'blue'),  # hip - right hip
    ((1, 2), 'blue'),  # right hip - right knee
    ((2, 3), 'blue'),  # right knee - right foot
    ((14, 15), 'blue'),  # right shoulder - right elbow
    ((15, 16), 'blue'),  # right elbow - right wrist
    ((14, 8), 'blue'),  # right shoulder - thorax

    # Middle (black)
    ((8, 7), 'gray'),  # thorax - spine
    ((8, 9), 'gray'),  # thorax - neck/nose
    ((9, 10), 'gray'),  # neck/nose - head
    ((7, 0), 'gray')   # spine - hip
]

def draw_colored_skeleton(image_path, keypoints, output_dir):
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Plot the skeleton with specified colors
    for (start, end), color in skeleton_connections:
        # Get coordinates of the starting and ending keypoints
        x_start, y_start = keypoints[start]
        x_end, y_end = keypoints[end]

        # Define color in RGB for cv2 (convert from string to BGR format)
        color_rgb = {
            'green': (0, 255, 0),
            'blue': (0, 0, 255),
            'gray': (128, 128, 128)
        }[color]

        # Draw lines between keypoints to form the skeleton
        cv2.line(image_rgb, (int(x_start), int(y_start)), (int(x_end), int(y_end)), color_rgb, 10)

    # Plot keypoints as circles
    for (x, y) in keypoints:
        cv2.circle(image_rgb, (int(x), int(y)), 10, (255, 0, 0), -1)

    # Convert the image back to BGR format to save with OpenCV
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # Construct the output file path
    cv2.imwrite(output_dir, image_bgr)

    print(f"Image saved to {output_dir}")

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
#We are using the pre-trained Detectron2 model, as shown below.
cfg = get_cfg()

cfg.MODEL.DEVICE = "cuda"
# load the pre trained model from Detectron2 model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
# set confidence threshold for this model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
# load model weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml")
# create the predictor for pose estimation using the config

In [ ]:
import numpy as np
import glob

predictor = DefaultPredictor(cfg)

In [ ]:
def create_folder(directory_path):
    # Check if the directory already exists
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' created.")
    else:
        print(f"Directory '{directory_path}' already exists.")

In [ ]:
def coco2h36m(x):
    '''
        Input: x (M x T x V x C)

        COCO: {0-nose 1-Leye 2-Reye 3-Lear 4Rear 5-Lsho 6-Rsho 7-Lelb 8-Relb 9-Lwri 10-Rwri 11-Lhip 12-Rhip 13-Lkne 14-Rkne 15-Lank 16-Rank}

        H36M:
        0: 'root',
        1: 'rhip',
        2: 'rkne',
        3: 'rank',
        4: 'lhip',
        5: 'lkne',
        6: 'lank',
        7: 'belly',
        8: 'neck',
        9: 'nose',
        10: 'head',
        11: 'lsho',
        12: 'lelb',
        13: 'lwri',
        14: 'rsho',
        15: 'relb',
        16: 'rwri'
    '''
    y = np.zeros(x.shape)
    y[:,0,:] = (x[:,11,:] + x[:,12,:]) * 0.5
    y[:,1,:] = x[:,12,:]
    y[:,2,:] = x[:,14,:]
    y[:,3,:] = x[:,16,:]
    y[:,4,:] = x[:,11,:]
    y[:,5,:] = x[:,13,:]
    y[:,6,:] = x[:,15,:]
    y[:,8,:] = (x[:,5,:] + x[:,6,:]) * 0.5
    y[:,7,:] = (y[:,0,:] + y[:,8,:]) * 0.5
    y[:,9,:] = x[:,0,:]
    y[:,10,:] = (x[:,1,:] + x[:,2,:]) * 0.5
    y[:,11,:] = x[:,5,:]
    y[:,12,:] = x[:,7,:]
    y[:,13,:] = x[:,9,:]
    y[:,14,:] = x[:,6,:]
    y[:,15,:] = x[:,8,:]
    y[:,16,:] = x[:,10,:]
    return y

In [ ]:
actions = ['courtyard_basketball_01', 'courtyard_jumpBench_01' , 'outdoors_parcours_00', 'outdoors_climbing_02', 'downtown_enterShop_00', 'downtown_stairs_00', 'flat_guitar_01', 'flat_packBags_00']

In [ ]:
directory = '/imageFiles/'
output_directory = '/outputs/'

import os

for action in actions:
    vid_dir = directory + action + '/*.jpg'
    create_folder(output_directory + action)
    img_files = [f for f in glob.glob(vid_dir) if os.path.isfile(f)]
    keypoints_per_cam = []
    for img_dir in sorted(img_files):
        file_name = os.path.basename(img_dir)
        output_dir = output_directory + action + '/' + file_name
        im = cv2.imread(img_dir)
        outputs = predictor(im)
        predicted = outputs["instances"].pred_keypoints.cpu().numpy()
        if len(predicted) > 0:
            predicted = coco2h36m(predicted)[0][:, :2]
        else:
            predicted = keypoints_per_cam[-1]
            
        result_image = draw_colored_skeleton(img_dir, predicted, output_dir)